<a href="https://colab.research.google.com/github/vgomesfalcao/planilha_central/blob/main/Capital_planilha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', None)

In [ ]:
path = 'itau_convertido.xlsx'
razao = pd.read_excel(path)
razao = razao.drop(range(8))
razao = razao.drop(razao.index[-1])
razao = razao.dropna(how='all', axis=1)
razao.columns = ['Data', 'Lançamento','Contra-Partida','Parenteses','Atalho','Histórico','Valor de Débito','Valor de Crédito', 'Saldo','Deb/Cred']
razao['Atalho'] = razao['Parenteses'] + razao['Atalho']
razao = razao.drop(columns=['Parenteses'])
razao = razao.reset_index(drop=True)

In [ ]:
for row in razao.itertuples():
  if(pd.isnull(row.Data)):
    # display(row.Index-1)
    # break
    subindex = 1
    while True:
      if((row.Index-subindex) in razao.index):
        linha_anterior = razao.loc[[row.Index-subindex],'Histórico'].values[0]
        break
      else:
        subindex += 1
        
    razao.loc[[row.Index-subindex],'Histórico'] =  linha_anterior + row.Histórico
    razao = razao.drop(row.Index)
filtro = razao['Data']!='CENTRAL ORGANIZACAO CONTABIL S/C LTDA'
razao['Data'] = razao[filtro]
razao = razao.dropna(how='all', axis=0)
razao = razao.reset_index(drop=True)

In [ ]:
razao['Cheques'] = razao['Histórico'].str.extract(r'((?<=CH:)[\d,\s]{3,10})')
razao['Cheques'] = razao['Cheques'].str.replace(' ','')
razao

,Data,Lançamento,Contra-Partida,Atalho,Histórico,Valor de Débito,Valor de Crédito,Saldo,Deb/Cred,Cheques
0,03/05/2021,41585.0,2.117020e+09,(94),CH 10851,NaN,760.50,1638156.13,D,NaN
1,03/05/2021,41586.0,2.117020e+09,(94),CH 10493,NaN,1291.50,1636864.63,D,NaN
2,03/05/2021,41587.0,2.117020e+09,(94),CH 10496,NaN,1340.00,1635524.63,D,NaN
3,03/05/2021,41588.0,2.117020e+09,(94),CH 11190,NaN,1601.50,1633923.13,D,NaN
4,03/05/2021,41589.0,2.117020e+09,(94),CH 11219,NaN,2396.86,1631526.27,D,NaN
...,...,...,...,...,...,...,...,...,...,...
2733,31/05/2021,45226.0,1.121010e+09,(1060),B-9193-1 CH:61004,44005.00,NaN,2935307.69,D,61004
2734,31/05/2021,45227.0,1.125010e+09,(383),61073-1 CH:12014,NaN,765.00,2934542.69,D,12014
2735,31/05/2021,45228.0,1.125010e+09,(383),61080-1 CH:12027,NaN,736.50,2933806.19,D,12027
2736,31/05/2021,45229.0,1.121010e+09,(1046),B-9141-1 CH:60963,38863.71,NaN,2972669.90,D,60963


In [ ]:
datatoexcel = pd.ExcelWriter('processado/cleaned_'+path)
razao.to_excel(datatoexcel)
datatoexcel.save()

In [ ]:
sygecom = pd.read_excel('Planilha Excel Sygecom.xlsx')
sygecom = sygecom.drop(columns=['Data Emi','Data Comp','Unnamed: 9','Pago/Rec P/Usuário','Unnamed: 18','Hora Lanc','Filial','Favorecido','Contato','Conta DEB','Conta CRE','CPF/CNPJ','Unnamed: 30','Invoice','Contrato','Adiantamento','Cód. Baixa'])
sygecom.rename(columns={'Data Venc':'Data','Saldo   R$':'Saldo','Cheque N.':'Cheques'},inplace = True)
sygecom = sygecom.dropna(how='all', axis=1)

In [ ]:
razaona = razao
razaona[['Cheques']] = razaona[['Cheques']].fillna(0)
razaona['Cheques'] = razaona['Cheques'].astype('int64')
razaona['Data'] = pd.to_datetime(razao['Data'],format='%d/%m/%Y')


In [ ]:
sygecom['Cheques'] = sygecom['Cheques'].astype('int64')
sygecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1272 entries, 0 to 1271
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Data                    1272 non-null   datetime64[ns]
 1   Documento               1272 non-null   object        
 2   Descricao da Conta      1272 non-null   object        
 3   Cheques                 1272 non-null   int64         
 4   Entrada R$              1272 non-null   float64       
 5   Saida   R$              1272 non-null   float64       
 6   Saldo                   1272 non-null   float64       
 7   Descricao do Historico  1272 non-null   object        
 8   Observações             1272 non-null   object        
 9   Nota Fiscal             1272 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 99.5+ KB


In [ ]:
razaona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2738 entries, 0 to 2737
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Data              2738 non-null   datetime64[ns]
 1   Lançamento        2738 non-null   float64       
 2   Contra-Partida    2738 non-null   float64       
 3   Atalho            2738 non-null   object        
 4   Histórico         2738 non-null   object        
 5   Valor de Débito   569 non-null    float64       
 6   Valor de Crédito  2169 non-null   float64       
 7   Saldo             2738 non-null   float64       
 8   Deb/Cred          2738 non-null   object        
 9   Cheques           2738 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 214.0+ KB


In [ ]:
juncao = razaona.append(sygecom).sort_values(['Data','Cheques'])
datatoexcel = pd.ExcelWriter('processado/cleaned_juncao.xlsx')
juncao.to_excel(datatoexcel)
datatoexcel.save()